In [ ]:
# Import needed modules

import qiskit
import pqca
import midiutil
import math
import music21
import random

In [ ]:
# Setting up the initial matrix and the starting tessellations

initial_state = [0] * 9 * 4
tes_1 = pqca.tessellation.n_dimensional([9,4],[1,2])
tes_2 = pqca.tessellation.n_dimensional([9,4],[3,1])

cell_circuit = qiskit.QuantumCircuit(2)

# Adds a Hadamard port, which creates a sovrapposition between quantum states |0> and |1>
cell_circuit.h(0)

# Adds a CNOT port between qubit 0 and 1, which creates entanglement switching the current state of the ports
cell_circuit.cx(0, 1)

# Adds a Z port, which negates the value bit (0 => 0, 1 => -1)
cell_circuit.z(1)

cell_circuit.cx(0, 1)

# T and Ry ports are useful to set a rotation angle for the qubit
cell_circuit.t(1)
cell_circuit.ry(qiskit.circuit.Parameter('π'), 1)

# Adds a SWAP port
cell_circuit.swap(0, 1)

print(cell_circuit.draw())

cell_circuit2 = qiskit.QuantumCircuit(3)

cell_circuit2.h(0)

cell_circuit2.cx(1, 2)

cell_circuit2.z(1)

cell_circuit2.cx(0, 1)

cell_circuit2.t(2)

cell_circuit2.ry(qiskit.circuit.Parameter('π/4'), 2)

cell_circuit2.swap(0, 2)

print(cell_circuit2.draw())

In [ ]:
# Generates the quantum circuit, based on the tessellations and states we just created.

update_frame_1 = pqca.UpdateFrame(tes_1, qiskit_circuit = cell_circuit)
update_frame_2 = pqca.UpdateFrame(tes_2,  qiskit_circuit = cell_circuit2)
automaton = pqca.Automaton(initial_state, [update_frame_1, update_frame_2], pqca.backend.qiskit())
automaton.update_circuit.draw()


In [ ]:
# Generates two matrixs based on the new Quantum Circuit 

class Qubit_Generator:
    def __next__(self):
        thirty_six_bits = next(automaton)
        return [[thirty_six_bits[i+j] for j in range(9)] for i in range(4)]
QBG = Qubit_Generator()

bits = next(QBG)

def generate_multiple_bits(num_iterations):
    return [next(QBG) for _ in range(num_iterations)]
 
bits_sequence = generate_multiple_bits(4)

# Default possible instrumentation, will be replaced later.
instrumentation = (music21.instrument.ElectricGuitar, music21.instrument.ElectricPiano)

In [ ]:
# Splits the matrix into groups: the first column will determine the fundamental frequency,
# 2-5 will determine the triads and 6-9 will determine the possible instruments
def bit_breakdown(four_by_nine_bits):
    return {"fundamental": [row[0] for row in four_by_nine_bits],
            "triads": [row[1:5] for row in four_by_nine_bits],
            "instruments": [row[5:9] for row in four_by_nine_bits],
    }

# Notes in triads become different parts of the score
def triads_to_voice_parts(triad_data):
    voices = [[],[],[]]
    for index, triad in enumerate(triad_data):
        for voice in [0,1,2]:
            voices[voice].append({
                "pitch": triad["intervals"][voice],
                "entry": index*3 + triad["timings"]["entry"][voice],
                "exit": (index+1)*3 - triad["timings"]["exit"][voice],
                "instrument": triad["instruments"][voice]
            })
    return voices

def breakdown_to_semitone_shift(bits):
    return int("".join(map(str, bits)),2)

def triad_bits_to_coordinate_sequence(triad_bits):
    triads = []
    for j in range(0, len(triad_bits)):
        for i in range(0,len(triad_bits[j])):
            if triad_bits[j][i]:
                triads.append([i,j])
    return triads

def intervals_to_triad(fundamental, coordinate): # Function has been modified in order to add variety and add the chance to generate major intervals
    notes = []
    notes.append(fundamental)
    notes.append(notes[0] + coordinate[0])
    if random.choice([True, False]):
        notes.append(notes[1] + coordinate[1])
    else:
        notes.append(notes[1] + coordinate[1] + 1)  # randomly increase interval by a semitone
    return notes

# This function randomly adjusts the intervals, garanting more balance in the score.
def balance_intervals(triad_data):
    for triad in triad_data:
        if random.random() < 0.5:
            if triad["intervals"][2] - triad["intervals"][1] == 3:
                triad["intervals"][2] += 1
    return triad_data

# Lookup, Timings are Neighbours are essential to later apply the CAMUS ruleset for cellular automata.
def lookup(bit_grid, coordinates):
    if coordinates[1] < 0 or coordinates[1] >= len(bit_grid):
        return 0
    if coordinates[0] < 0 or coordinates[0] >= len(bit_grid[0]):
        return 0
    return bit_grid[coordinates[1]][coordinates[0]]

def timings(bits, triad_coord, short_wait=0.5, long_wait=1):
    n = neighbours(bits, [triad_coord[0]+1, triad_coord[1]])
    entry = [n[direction] * (short_wait if n["d"] else long_wait) for direction in "abc"]
    exit = [n[direction] * (short_wait if n["p"] else long_wait) for direction in "mno"]
    return {"entry": entry, "exit": exit }

def neighbours(bit_grid, coordinate):

    x_mid = coordinate[0]
    y_mid = coordinate[1]
    ordered = [lookup(bit_grid,[x,y]) 
            for y in (y_mid-1, y_mid, y_mid+1)
            for x in (x_mid-1, x_mid, x_mid+1)
           ]
    return {
        "a": ordered[1],
        "b": ordered[7],
        "c": ordered[5],
        "d": ordered[3],
        "m": ordered[0],
        "n": ordered[8],
        "o": ordered[2],
        "p": ordered[6]
    }

def instrument_from_triad_coord(bits, triad_coord):
    instrument_coord = [triad_coord[0]+5, triad_coord[0]]
    n = neighbours(bits, instrument_coord)
    return [n["a"], n["b"], n["c"]]

def bits_to_triad_sequence(bits):
    breakdown = bit_breakdown(bits)
    fundamental = breakdown_to_semitone_shift(breakdown["fundamental"])
    triad_coordinates = triad_bits_to_coordinate_sequence(breakdown["triads"])
    triad_data = [{"intervals": intervals_to_triad(fundamental, coordinate),
                  "instruments": instrument_from_triad_coord(bits, coordinate),
                  "timings": timings(bits, coordinate)}
                  for coordinate in triad_coordinates] 
    return triad_data

def sequence_of_bit_grids_to_triads(list_of_bit_grids):
    return [triad 
            for bits in list_of_bit_grids
            for triad in bits_to_triad_sequence(bits)]

In [ ]:
# # OPTIONAL: Some more musical tweaks: we try to avoid undesirable off-scale notes, by snapping pitches to scale.
# We also want to avoid notes too short to be relevant at all.

def harmonic_attraction(pitch, center_pitch=60):
    distance = abs(pitch - center_pitch)
    attraction = 1 / (distance + 1)
    new_pitch = pitch + int((center_pitch - pitch) * attraction * 0.5)
    return snap_to_scale(new_pitch)

def snap_to_scale(pitch, scale=[0, 2, 4, 5, 7, 9, 11]):
    pitch_class = pitch % 12
    closest_scale_pitch = min(scale, key=lambda x: abs(x - pitch_class))
    return pitch - pitch_class + closest_scale_pitch

def ensure_minimum_duration(duration, minimum=0.125):
    return max(duration, minimum)

# OPTIONAL: We can force patterns' repetitions into our score, or try to add some quantization.

def quantize_time(time, grid=1):
    return round(time / grid) * grid

def repeat_pattern(notes, repetitions=2):
    return notes * repetitions

In [ ]:
# Finally, we can wrap up everything we've done in order to make our automatic composer.

def bit_grids_to_midi_instrumentation_from_CAMUS(list_of_bit_grids, base_octave, bpm, ts, instruments):
    midiFile = midiutil.MIDIFile(1)
    midiFile.addTempo(0, 0, bpm)

    numerator, denominator = ts
    midiFile.addTimeSignature(0, 0, numerator, int(math.log2(denominator)), 24, 8)
    
    triad_sequence = balance_intervals(sequence_of_bit_grids_to_triads(list_of_bit_grids))
    voice_parts = [repeat_pattern(part) for part in triads_to_voice_parts(triad_sequence)]
    
    base_pitch = 60  # let's assume C4 (MIDI = 60) is our base pitch
    octave_shift = (base_octave - 4) * 12

    for voice, voice_part in enumerate(voice_parts):
        for note in voice_part:
            # module operation is useful to guarantee that the note will still be in our chosen octave. We also don't want the note to be outside the MIDI pitch range.
            pitch = harmonic_attraction(((note["pitch"] % 12) + base_pitch + octave_shift) % 128) 
            instrument = instruments[note["instrument"]]().midiProgram
            entry = quantize_time(float(note["entry"]))
            duration = quantize_time(float(note["exit"] - note["entry"]))
            if duration > 0:
                midiFile.addProgramChange(0, voice, entry, instrument) 
                midiFile.addNote(0, voice, pitch, entry, duration, 100)
    
    with open('output\\pqca_2D.midi', "wb") as output_file:
        midiFile.writeFile(output_file)

In [ ]:
# CSS stylesheet for iPyWidgets to display something more visually interesting

custom_css = """
<style>
body {
    font-family: Helvetica, Arial, sans-serif;
}
h1 {
    text-align: center;
    color: #8a2be2;
    margin-bottom: 30px;
}
.widget-dropdown {
    width: 200px !important;
    margin-bottom: 15px !important;
}
.widget-dropdown > select {
    background-color: #f0e6ff;
    border: 1px solid #d8b6ff;
    border-radius: 5px;
    text-align-last: center;
}
.widget-slider {
    width: 300px !important;
    margin-bottom: 15px !important;
    border-color: #8a2be2 !important;
}
.widget-slider .ui-slider-handle {
    background-color: #9370db !important;
    foreground-color: #8a2be2 !important;
}
.widget-slider .ui-slider-handle {
    background-color: #8a2be2 !important;
    border-color: #8a2be2 !important;
}

.widget-slider:hover .ui-slider-handle {
    background-color: #9370db !important;
}
.widget-button {
    background-color: #8a2be2 !important;
    color: white !important;
    border: none !important;
    border-radius: 5px !important;
    padding: 10px 20px !important;
    font-weight: bold !important;
    margin-top: 10px !important;
    display: flex !important;
    align-items: center !important;
    justify-content: center !important;
}
.widget-button:hover {
    background-color: #9370db !important;
}
.widget-label {
    text-align: center !important;
}
</style>
"""

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import music21

def get_instrument_list():
    return sorted([i.lower() for i in dir(music21.instrument) if i[0].isupper()])

display(HTML(custom_css))
display(HTML("<h1>JaQo - Just Another Quantum Orchestra</h1>"))

octave_widget = widgets.IntSlider(
    min=2, max=6, value=4,
    description='Octave:',
    style={'description_width': 'initial'}
)

bpm_widget = widgets.IntSlider(
    min=60, max=240, value=120,
    description='Tempo:',
    style={'description_width': 'initial'}
)

ts_num_widget = widgets.Dropdown(
    options=[2, 3, 4, 5, 6, 7, 8, 9, 12],
    value=4,
    description='Time Numerator:',
    style={'description_width': 'initial'}
)

ts_den_widget = widgets.Dropdown(
    options=[2, 4, 8, 16, 24, 32],
    value=4,
    description='Time Denominator:',
    style={'description_width': 'initial'}
)

# Possible instruments: keep in mind that, for some reason, some instruments don't work at all. Will eventually try to fix.

instrument1_widget = widgets.Dropdown(
    options=get_instrument_list(),
    value='guitar',
    description='Possible Instrument 1:',
    style={'description_width': 'initial'}
)

instrument2_widget = widgets.Dropdown(
    options=get_instrument_list(),
    value='trumpet',
    description='Possible Instrument 2:',
    style={'description_width': 'initial'}
)

instrument3_widget = widgets.Dropdown(
    options=get_instrument_list(),
    value='tenor',
    description='Possible Instrument 3:',
    style={'description_width': 'initial'}
)

generate_button = widgets.Button(description='Generate')

output = widgets.Output()

layout = widgets.VBox([
    widgets.HBox([
        widgets.VBox([bpm_widget, instrument1_widget]),
        widgets.VBox([ts_num_widget, instrument2_widget, instrument3_widget]),
        widgets.VBox([octave_widget, ts_den_widget])
    ], layout=widgets.Layout(justify_content='space-between', align_items='flex-start')),
    widgets.HBox([generate_button], layout=widgets.Layout(justify_content='center', margin='10px 0 0 0')),
    output
], layout=widgets.Layout(width='100%', padding='20px'))

def on_generate_button_clicked(b):
    with output:
        output.clear_output()
        print("Generating...")
        
        octave = octave_widget.value
        bpm = bpm_widget.value
        ts = (ts_num_widget.value, ts_den_widget.value)
        instruments = [getattr(music21.instrument, instrument1_widget.value.capitalize()),
                       getattr(music21.instrument, instrument2_widget.value.capitalize()),
                       getattr(music21.instrument, instrument3_widget.value.capitalize())]
        
        bits_sequence = generate_multiple_bits(4)
        
        bit_grids_to_midi_instrumentation_from_CAMUS(bits_sequence, octave, bpm, ts, instruments)
        
        print("MIDI file generated: pqca_2D.midi")
        
        print("\nQuantum circuit:")
        display(automaton.update_circuit.draw())

generate_button.on_click(on_generate_button_clicked)
display(layout)